## Prepare initial snapshots for CG, Rigid and AA

In [2]:
import gsd.hoomd
import numpy as np

In [4]:
kT_list = [4.0, 7.0]

## Initial Rigid Snapshot kT=4.0

In [5]:
initial_rigid_snap = gsd.hoomd.open("/home/marjanalbooyeh/Aniso_ML_MD_project/Aniso_MLMD/aniso_MLMD/sampling/logs/pps_200/trajectory_4.0.gsd")[-1]

In [6]:
initial_rigid_snap.particles.types

['rigid', 'ca', 'sh']

In [7]:
with gsd.hoomd.open("init_rigid_4.0.gsd", "w") as traj:
    traj.append(initial_rigid_snap)

## Initial CG snapshot kT=4.0

In [8]:
rigid_ids = np.where(initial_rigid_snap.particles.typeid==0)[0]
N_beads = rigid_ids.shape[0]
cg_snapshpot = gsd.hoomd.Frame()
cg_snapshpot.particles.types = ["A"]
cg_snapshpot.particles.N = N_beads
cg_snapshpot.particles.typeid = [0] * N_beads

cg_snapshpot.particles.mass = initial_rigid_snap.particles.mass[:N_beads]
cg_snapshpot.particles.position = initial_rigid_snap.particles.position[:N_beads]
cg_snapshpot.particles.orientation = initial_rigid_snap.particles.orientation[:N_beads]
cg_snapshpot.particles.moment_inertia = initial_rigid_snap.particles.moment_inertia[:N_beads]
cg_snapshpot.particles.velocity = initial_rigid_snap.particles.velocity[:N_beads]

cg_snapshpot.configuration.box = initial_rigid_snap.configuration.box
with gsd.hoomd.open("init_cg_4.0.gsd", "w") as traj:
    traj.append(cg_snapshpot)

In [10]:
cg_snapshpot.particles.types

['A']

## Initial UA snapshot kT=4.0

In [11]:
from flowermd.library import PPS, OPLS_AA_PPS
from flowermd.base import Pack
pps = PPS(num_mols=N_beads, lengths=1)
system = Pack(molecules=pps, density=0.9, packing_expand_factor=7)
system.apply_forcefield(r_cut=2.5, force_field=OPLS_AA_PPS(), auto_scale=True,
                        remove_charges=True, scale_charges=True,
                        remove_hydrogens=True)

/home/marjanalbooyeh/miniconda3/envs/anisoML/lib/python3.11/site-packages/mdtraj/formats/__init__.py:6: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from .xtc import XTCTrajectoryFile
/home/marjanalbooyeh/miniconda3/envs/anisoML/lib/python3.11/site-packages/mbuild/recipes/__init__.py:13: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  entry_points = metadata.entry_points()["mbuild.plugins"]
/home/marjanalbooyeh/Aniso_ML_MD_project/Aniso_MLMD_March2024/flowerMD/flowermd/base/system.py:659: UserWarning: Units for density were not given, assuming units of g/cm**3.
  warnings.warn(
/home/marjanalbooyeh/miniconda3/envs/anisoML/lib/python3.11/site-packages/gmso/core/atom_type.py:198: UserWarning: Masses are assumed to be in units of g/mol
  warnings.warn(UNIT_WARNING_STRING.format("Masses", "g/mol"))
/home/marjanalbooyeh/miniconda3/envs/anisoML/lib/python3.11/site-packages/gmso/core/atom_type.py:209: UserWarning: Charges 

No charged group detected, skipping electrostatics.


/home/marjanalbooyeh/miniconda3/envs/anisoML/lib/python3.11/site-packages/gmso/external/convert_hoomd.py:1247: UserWarning: RyckaertBellemansTorsionPotential will be converted to OPLSTorsionPotential.
  warnings.warn(
/home/marjanalbooyeh/miniconda3/envs/anisoML/lib/python3.11/site-packages/gmso/external/convert_hoomd.py:562: UserWarning: Orthorhombic box detected
  warnings.warn("Orthorhombic box detected")
/home/marjanalbooyeh/miniconda3/envs/anisoML/lib/python3.11/site-packages/gmso/external/convert_hoomd.py:117: NotYetImplementedWarning: Only writing particle, bond, sangle, proper and improper dihedral information.Special pairs are not currently written to GSD files
  warnings.warn(
/home/marjanalbooyeh/miniconda3/envs/anisoML/lib/python3.11/site-packages/gmso/external/convert_hoomd.py:274: UserWarning: Shifting coordinates to [-L/2, L/2]
  warnings.warn("Shifting coordinates to [-L/2, L/2]")
/home/marjanalbooyeh/miniconda3/envs/anisoML/lib/python3.11/site-packages/gmso/external/co

In [12]:
ua_snapshot = system.hoomd_snapshot

In [14]:
ua_snapshot.configuration.box

array([66.86016111, 66.86016111, 66.86016111,  0.        ,  0.        ,
        0.        ])

In [15]:
ua_snapshot.particles.position = initial_rigid_snap.particles.position[N_beads:]
ua_snapshot.particles.velocity = initial_rigid_snap.particles.velocity[N_beads:]
ua_snapshot.particles.orientation = initial_rigid_snap.particles.orientation[N_beads:]
ua_snapshot.configuration.box = initial_rigid_snap.configuration.box

In [16]:
ua_snapshot.configuration.box

array([9.551452, 9.551452, 9.551452, 0.      , 0.      , 0.      ],
      dtype=float32)

In [17]:
ua_snapshot.particles.types

['ca', 'sh']

In [18]:
with gsd.hoomd.open("init_UA_4.0.gsd", "w") as traj:
    traj.append(ua_snapshot)

In [19]:
pps_ff = system.hoomd_forcefield
system.pickle_forcefield("pps_ff.pkl")

## Information

In [28]:
aa_frame = gsd.hoomd.open("/home/marjanalbooyeh/Aniso_ML_MD_project/Aniso_MLMD/aniso_MLMD/sampling/assets/pps_aa_N200.gsd")[0]
positions = aa_frame.particles.position
# indices of constituent particles
constituent_indx = [np.arange(7) + (i*7) for i in range(N_beads) ]


com_mass = []
com_positions = []

for indx in constituent_indx:
    total_mass = np.sum(aa_frame.particles.mass[indx])
    com_mass.append(total_mass)
    com_positions.append(
                np.sum(
                    aa_frame.particles.position[indx]
                    * aa_frame.particles.mass[indx, np.newaxis],
                    axis=0,
                )
                / total_mass
            )


rel_const_pos =  positions[constituent_indx[0]] - com_positions[0]
rel_const_pos

array([[ 0.04237747, -0.02183342, -0.64930916],
       [ 0.07262421, -0.34874725, -0.44158173],
       [ 0.04734802, -0.33596802, -0.05404472],
       [-0.00818253,  0.00425148,  0.12626076],
       [-0.04059601,  0.02091789,  0.62210274],
       [-0.03852081,  0.33138275, -0.08208847],
       [-0.01317978,  0.3181038 , -0.4696045 ]], dtype=float32)

In [29]:
const_particle_types = list(np.asarray(aa_frame.particles.types)[list(aa_frame.particles.typeid)][constituent_indx[0]])

In [30]:
const_particle_types

['ca', 'ca', 'ca', 'ca', 'sh', 'ca', 'ca']

In [31]:
np.save("rel_const_pos.npy", rel_const_pos)

In [34]:
import pickle
with open("assets/pps_ff.pkl", "rb") as f:
    ff = pickle.load(f)



In [36]:
gsd.hoomd.open("assets/init_UA_4.0.gsd")[0]